# リスト内包表記とドット演算子の速度比較

In [1]:
using MyTools, BenchmarkTools

In [2]:
w = randn(1000)   #Soft-Thresholding適用対象の配列
w2 = zeros(1000)  #結果格納用（メモリ確保の時間を除くため）
l = 0.1           #Soft-Thresholdingの閾値
w'

1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.22328  -0.96827  1.42927  1.29484  …  0.0105219  0.821605  -0.0686189

**リスト内包表記でSoft-Thresholding1**

In [3]:
@btime w2 .= [wi > l ? (wi - l) : wi < -l ? (wi + l) : 0.0 for wi in w]
w2'

  99.099 μs (3976 allocations: 70.05 KiB)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.12328  -0.86827  1.32927  1.19484  0.41411  …  1.62926  0.0  0.721605  0.0

**リスト内包表記でSoft-Thresholding2**

In [4]:
@btime w2 .= [abs(wi) > l ? sign(wi) * (abs(wi) - l) : 0.0 for wi in w]
w2'

  91.999 μs (4640 allocations: 80.42 KiB)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.12328  -0.86827  1.32927  1.19484  0.41411  …  1.62926  0.0  0.721605  0.0

**ドット演算子でSoft-Thresholding1**

In [5]:
@btime @. w2 = (w > l) * (w - l) + (w < -l) * (w + l)
w2'

  5.316 μs (16 allocations: 368 bytes)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.12328  -0.86827  1.32927  1.19484  0.41411  …  1.62926  0.0  0.721605  0.0

**ドット演算子でSoft-Thresholding2**

In [6]:
@btime @. w2 = (abs(w) > l) * sign(w) * (abs(w) - l)
w2'

  3.087 μs (12 allocations: 240 bytes)


1×1000 LinearAlgebra.Adjoint{Float64,Array{Float64,1}}:
 1.12328  -0.86827  1.32927  1.19484  …  1.62926  -0.0  0.721605  0.0